In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error

In [26]:
# Đọc dữ liệu
df = pd.read_csv(r'C:\Users\DELL\Downloads\file_tieuluan2\QLproduct_.csv')
df

,timestamp,name,price,price_min,price_max,price_min_before_discount,price_max_before_discount,itemid
0,2023-06-18 16:44:47,"Túi nước giặt OMO Matic 3,6kg/ 3,9Kg",17950000000,17950000000,17950000000,22700000000,22700000000,2.820088e+09
1,2023-06-18 16:44:47,Nước xả làm mềm vải Comfort Chăm sóc Chuyên sâ...,20900000000,20900000000,21100000000,27100000000,27100000000,5.657384e+09
2,2023-06-18 16:44:47,Nước rửa chén sunlight chanh mới sạch nhanh xả...,6200000000,6200000000,6200000000,6500000000,6500000000,7.517756e+09
3,2023-06-18 16:44:47,Nước Xả Vải Em Bé Comfort Cho Da Nhạy Cảm Túi ...,21900000000,21900000000,21900000000,28500000000,28500000000,3.044711e+09
4,2023-06-18 16:44:47,Nước Rửa Chén Bát Sunlight Chanh 100 Sạch Nhan...,9900000000,9900000000,9900000000,10490000000,10490000000,4.539086e+08
...,...,...,...,...,...,...,...,...
262,2023-06-18 16:44:55,Combo 2 túi Nước Giặt Omo Matic 2.0kgx2,21900000000,21900000000,21900000000,26400000000,26400000000,5.332837e+09
263,2023-06-18 16:44:55,[HC GIFT] Túi nước xả vải Comfort 200ml-Hương ...,50000000000,50000000000,50000000000,-1,-1,1.868881e+10
264,2023-06-18 16:44:55,Combo 6 Hộp Viên Rửa Chén Sunlight 5 in 1 Dành...,59500000000,59500000000,59500000000,84000000000,84000000000,5.262396e+09
265,2023-06-18 16:44:55,Combo 6 Túi Viên Giặt Tiện Lợi OMO Công Nghệ A...,63900000000,63900000000,63900000000,84000000000,84000000000,1.225640e+10


In [4]:
# Chuyển đổi cột timestamp thành định dạng datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Chọn dữ liệu cần thiết
data = df[['timestamp', 'name', 'price']]
data.set_index('timestamp', inplace=True)

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data[['price']])
#



In [5]:
# Tạo các tập dữ liệu huấn luyện
def create_dataset(dataset, look_back=50):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:i+look_back, 0])
        Y.append(dataset[i+look_back, 0])
    return np.array(X), np.array(Y)

look_back = 50
X, y = create_dataset(scaled_data, look_back)

# Chuyển đổi dữ liệu cho LSTM
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [7]:
# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(units=64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam')

In [8]:
# Lưu mô hình tốt nhất
checkpoint = ModelCheckpoint('best_model.keras', monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(X, y, epochs=10, batch_size=32, verbose=2, callbacks=[checkpoint])

Epoch 1/10

Epoch 1: loss improved from inf to 0.21933, saving model to best_model.keras
7/7 - 12s - 2s/step - loss: 0.2193
Epoch 2/10

Epoch 2: loss improved from 0.21933 to 0.20936, saving model to best_model.keras
7/7 - 1s - 119ms/step - loss: 0.2094
Epoch 3/10

Epoch 3: loss did not improve from 0.20936
7/7 - 1s - 97ms/step - loss: 0.2094
Epoch 4/10

Epoch 4: loss improved from 0.20936 to 0.19531, saving model to best_model.keras
7/7 - 1s - 102ms/step - loss: 0.1953
Epoch 5/10

Epoch 5: loss did not improve from 0.19531
7/7 - 1s - 88ms/step - loss: 0.2054
Epoch 6/10

Epoch 6: loss did not improve from 0.19531
7/7 - 1s - 85ms/step - loss: 0.1985
Epoch 7/10

Epoch 7: loss did not improve from 0.19531
7/7 - 1s - 89ms/step - loss: 0.2022
Epoch 8/10

Epoch 8: loss did not improve from 0.19531
7/7 - 1s - 88ms/step - loss: 0.2009
Epoch 9/10

Epoch 9: loss did not improve from 0.19531
7/7 - 1s - 85ms/step - loss: 0.1998
Epoch 10/10

Epoch 10: loss did not improve from 0.19531
7/7 - 1s - 10

In [9]:
# Tải mô hình tốt nhất
model = load_model('best_model.keras')


In [10]:
# Tạo scaler và fit với dữ liệu huấn luyện
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(data[['price']])

# Dự đoán giá tương lai
future_days = 10
future_predictions = []

last_look_back = scaled_data[-look_back:]

for _ in range(future_days):
    next_pred = model.predict(last_look_back.reshape(1, look_back, 1))
    future_predictions.append(next_pred)
    last_look_back = np.append(last_look_back[1:], next_pred).reshape(look_back, 1)

# Chuyển đổi giá trị dự đoán về giá trị thực
future_predictions = np.array(future_predictions).reshape(-1, 1)
future_predictions = scaler.inverse_transform(future_predictions).flatten()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


In [12]:
current_price = data['price'].values[-1]
recommendations = []

# Tạo danh sách các sản phẩm và giá dự đoán
for day, pred_price in enumerate(future_predictions):
    if pred_price < current_price:
        recommendations.append((data['name'].iloc[day], pred_price))

# Sắp xếp danh sách theo giá dự đoán
recommendations.sort(key=lambda x: x[1])

if recommendations:
    print("Các sản phẩm bạn nên xem xét mua:")
    for i, (product, pred_price) in enumerate(recommendations[:6], 1):
        print(f"Top {i}: {product} - Giá dự đoán: {pred_price:.2f} VND")
else:
    print('Không có sản phẩm nào được khuyến nghị, giá dự đoán không thấp hơn giá hiện tại.')


Các sản phẩm bạn nên xem xét mua:
Top 1: Combo 2 Túi Nước giặt OMO Matic chuyên dụng Cửa Trên 3.9kg - Giá dự đoán: 22803802112.00 VND
Top 2: Combo 2 Nước Tẩy Bồn Cầu Vim Diệt Khuẩn (880ml/chai) - Xanh Biển - Giá dự đoán: 23409178624.00 VND
Top 3: Combo 2 Túi Nước giặt OMO Matic chuyên dụng Cửa Trước 3.6kg/túi - Giá dự đoán: 24037836800.00 VND
Top 4: Viên Treo Bồn Cầu VIM POWER 5 sạch khuẩn thơm mát 55g/viên - Giá dự đoán: 24674398208.00 VND
Top 5: Nước Lau Sàn Sunlight Tinh Dầu Thảo Mộc Túi 3.6 Kg - Giá dự đoán: 25293842432.00 VND
Top 6: Nước Rửa Chén Bát Sunlight Chanh 100 Sạch Nhanh Xả Bọt Nhanh Rửa Sạch Dầu Mỡ 3.6kg - Giá dự đoán: 25858234368.00 VND


In [15]:
# Bước  9:  vẽ biểu đồ So sanh giá ban đầu và giá dự đoán
# Get the actual future prices from the dataset for comparison
actual_prices = data['price'].values[-(future_days + look_back):-look_back]

# Create a DataFrame to store actual and predicted prices
comparison_df = pd.DataFrame({
    'Actual Price': actual_prices,
    'Predicted Price': future_predictions
})

# Calculate performance metrics
r2 = r2_score(actual_prices, future_predictions)
mae = mean_absolute_error(actual_prices, future_predictions)
mape = mean_absolute_percentage_error(actual_prices, future_predictions)

# Print performance metrics
print(f"R2 Score: {r2}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")


R2 Score: -0.6838527425132443
Mean Absolute Error: 13505519731.2
Mean Absolute Percentage Error: 1.9938678279738142
